In [13]:
import tensorflow as tf
import numpy as np
from sklearn.neighbors import NearestNeighbors
import Nearest_Neighbor as nn

In [11]:
train_data_file = "D:/UCSD/F17/CSE293/data_prep_scripts/train_data.npy"
train_data = np.load(train_data_file)
test_data_file = "D:/UCSD/F17/CSE293/data_prep_scripts/test_data.npy"
test_data = np.load(test_data_file)

train_data_normalized_file = "D:/UCSD/F17/CSE293/data_prep_scripts/train_data_normalized.npy"
train_data_normalized = np.load(train_data_normalized_file)

test_data_normalized_file = "D:/UCSD/F17/CSE293/data_prep_scripts/test_data_normalized.npy"
test_data_normalized = np.load(test_data_normalized_file)

#row-wise normalizing make sure that the data is arranged rowwise
####### ran once to preprocess.

def normalize(X):
    X = (X / np.linalg.norm(X, axis=1)[:,None])
    return X

for lang_data in test_data: #each language
    for item_e in enumerate(lang_data): #for each example
        i, item = item_e
        lang_data[i] = normalize(item) #[[source], [target]]
        
np.save('D:/UCSD/F17/CSE293/data_prep_scripts/test_data_normalized.npy', test_data)

In [27]:
mlpm = model(ckpt_path="./ckpt/",lr=0.001)

start building graph
graph built


In [ ]:
#mlpm.train(train_data_normalized,train_lang_pairs= ["pt-es", "pt-en", "en-es"], batch_size=40, validation=[], test= test_data_normalized, test_lang_pairs=["pt-es"])
mlpm.train(train_data,train_lang_pairs= ["pt-es", "pt-en", "en-es"], batch_size=40, validation=[], test= test_data, test_lang_pairs=["pt-es"])

In [26]:
class model(object):
    def __init__(self, ckpt_path, lr, dim_lang=5, dim_model=300, model_name="MLPM"):
        #self.epochs = epochs
        self.ckpt_path = ckpt_path
        self.model_name = model_name
        self.lr = lr
        self.dim_lang = dim_lang
        self.dim_model = dim_model
        self.all_lang_rep = dict()
                
        for lang in ["en", "pt", "es"]:
            self.all_lang_rep[lang] = np.random.normal(loc=0.0, scale=1/np.sqrt(float(dim_lang)), size=(dim_lang))
        
        def __graph__():
            tf.reset_default_graph()
            self.graph = tf.Graph()
            with self.graph.as_default():
                
                # source and target vector representation
                self.source_words = tf.placeholder(tf.float32, shape=[None, dim_model])
                self.target_words = tf.placeholder(tf.float32, shape=[None, dim_model])
                
                # parameter matrices
                self.encoder = tf.Variable(tf.truncated_normal([dim_model, dim_model],\
                                           stddev=1/tf.sqrt(float(dim_model))), name='encoder')
                
                self.decoder = tf.Variable(tf.truncated_normal([dim_model, dim_model],\
                                           stddev=1/tf.sqrt(float(dim_model))), name='decoder')
                
                self.lang_encoder = tf.Variable(tf.truncated_normal([2*dim_lang,dim_model],\
                                               stddev=1/tf.sqrt(float(dim_model))), name='lang_encoder')

                # language representation
                self.lang_rep_placeholder = tf.placeholder(tf.float32, shape=[1,2*self.dim_lang])
                self.lang_rep = tf.Variable(tf.truncated_normal([1, 2* self.dim_lang],\
                                           stddev=1/tf.sqrt(2.0*dim_lang)))
                                
                self.assign_lang_rep_op = self.lang_rep.assign(self.lang_rep_placeholder)#.op 
                
                # model equation
                self.target_pred = self.get_model(self.encoder, self.decoder, self.source_words,\
                                                  self.lang_rep, self.lang_encoder)
                
                #squared loss
                self.loss = tf.reduce_sum(tf.square(self.target_words-self.target_pred))
                
                
                
                #self.train_step = tf.train.GradientDescentOptimizer(self.lr).minimize(self.loss)
                self.train_step = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)
                self.init = tf.global_variables_initializer()
                self.saver = tf.train.Saver()
        
        print('start building graph')
        __graph__()
        print('graph built')
        
    # encoder, decoder: 300x300   source_words: Nonex300    lang_rep: 1x2*dim_lang    lang_encoder: 2*dim_langx300
    # Output: Nonex300
    # we calculate the final projection in two parts  
    def get_model(self, encoder, decoder, source_words, lang_rep, lang_encoder):
        shared_source_words = tf.matmul(source_words, encoder)
        shared_lang_rep = tf.matmul(lang_rep, lang_encoder)
        
        num_examples = tf.shape(shared_source_words)[0]
        #(shared_source_words.get_shape())[0]
        tile = tf.tile(shared_lang_rep, tf.convert_to_tensor([1, num_examples]))
        shared_lang_aux = tf.reshape(tile, [num_examples, self.dim_model])
        shared_embedding_vector = shared_source_words + shared_lang_aux
        
        ret = tf.matmul(shared_embedding_vector, decoder)
        return ret 
    
    def get_feed(self, X, Y, src_lang, dest_lang):
        feed_dict = {self.source_words: X, self.target_words: Y}       
        return feed_dict
    
    #train -> train[en_pt]
    def train(self, train, train_lang_pairs, batch_size, validation, test, test_lang_pairs, \
              num_epochs=10, sess=None, log_file="./log_file.txt"):
        if sess == None:
            sess = tf.Session(graph = self.graph)
            sess.run(self.init)
            
        nearest_ne = nn.Nearest_Neighbor("D:/UCSD/F17/CSE293/", ["es"], 10)
                                         
        with sess.as_default():
            assert sess is tf.get_default_session()
            print (self.lang_rep.eval())
        #max data for a language pair/batch size
        max_data_size = int(max([len(x) for x in train])/batch_size)
        batch_counter_outer = 0
        with open(log_file, 'w') as log :
            log.write("Batch \t lang_pair \t loss\n")
            
            for epoch in range(num_epochs):
                batch_index = np.zeros(shape=(len(train)), dtype=int)
                for batch_number in range(max_data_size):
                    #make batches
                    batch_counter_outer += 1
                    
                    for lang_pair_data_enum in enumerate(train):
                        log_string = ""
                        i, lang_pair_data = lang_pair_data_enum
                        lang_pair_data = np.array(lang_pair_data)
                        
                        #rounding on the language pairs with less words in dictionary
                        if batch_index[i] >= len(lang_pair_data):
                            np.random.shuffle(train[i])
                            batch_index[i] = 0
            
                        cur_batch = lang_pair_data[batch_index[i]:(batch_index[i]+batch_size),:,:]
                        batch_index[i] += batch_size
            
                        X = cur_batch[:,0,:]
                        Y = cur_batch[:,1,:]
                        
                        lang = train_lang_pairs[i].split('-')

                        A = self.all_lang_rep[lang[0]]
                        B = self.all_lang_rep[lang[1]]
                        init = np.append(A,B)
                        
                        # making shape to (1, 2* dim_lang) from (, 2*dim_lang )
                        init = init.reshape(1, np.shape(init)[0])
                        
                        #assign the lang representation for a run
                        sess_lang_rep = sess.run(self.assign_lang_rep_op, feed_dict={self.lang_rep_placeholder : init})
                
                        #batch index more that lang pair data length not handled :P
                        _, train_loss, lang_rep, encoder, decoder, lang_encoder = \
                                    sess.run([self.train_step, self.loss, self.lang_rep, self.encoder, self.decoder, self.lang_encoder],\
                                             self.get_feed(X, Y, lang[0], lang[1]))
                        
                        log_string += str(batch_counter_outer) + " \t " + train_lang_pairs[i] + " \t " + str(train_loss/batch_size) + "\n"
                        log.write(log_string)
                    
                        self.all_lang_rep[lang[0]], self.all_lang_rep[lang[1]] = lang_rep[0][:self.dim_lang], lang_rep[0][self.dim_lang:]
                        
                        #test_code after every iteration  
                        self.test(test, test_lang_pairs, nearest_ne, sess)
                        #
                        #---------------------------------------------------------
                        #print("encoder : ", encoder)
                        #print("decoder : ", decoder)
                        #print("lang_encoder : ", lang_encoder)
                        #print("lang rep : ", lang_rep)
                        #print ("error : ", train_loss)
                        #if (lang[0] == 'pt'):
                        #    print("previous lang rep: ", A)
                        #    print("new lang rep :", lang_rep[0][:self.dim_lang])
                        
                        #print (lang[0], lang_rep[0][:5])
                        #print (lang[1], lang_rep[0][5:])
                        print("Batch:" + str(batch_counter_outer))
                        #-----------------------------------------------------------
                #save epoch
                if epoch and epoch%10==0:
                    self.saver.save(sess, self.ckpt_path+self.model_name+".ckpt", global_step=epoch)

                    #print losses #todo: format for train data with lang
        self.saver.save(sess, self.ckpt_path+self.model_name+".ckpt", global_step=num_epochs+1)
        #np.savetxt()
    
    def test(self, test, test_lang_pairs, nearest_neighbor, sess, log_file="./test_log_file.txt"):
        for lang_pair_data_enum in enumerate(test):
            i, lang_pair_data = lang_pair_data_enum
            lang_pair_data = np.array(lang_pair_data)
            
            lang = test_lang_pairs[i].split('-')
            A = self.all_lang_rep[lang[0]]
            B = self.all_lang_rep[lang[1]]
            
            init = np.append(A,B)
                        
            # making shape to (1, 2* dim_lang) from (, 2*dim_lang )
            init = init.reshape(1, np.shape(init)[0])

            #assign the lang representation for a run
            sess_lang_rep = sess.run(self.assign_lang_rep_op, feed_dict={self.lang_rep_placeholder : init})
            
            X = lang_pair_data[:,0,:]
            Y = lang_pair_data[:,1,:]
            #batch index more that lang pair data length not handled :P
            test_loss, target_pred = sess.run([self.loss, self.target_pred], self.get_feed(X, Y, lang[0], lang[1]))
            near_points = nearest_neighbor.knn(target_pred, lang[1], 10)
            self.calculate_precision(near_points, Y)
            print('--------------------')
            print(lang)
            print("Test loss: ", test_loss)
            
            '''print('P@1: ', nn(1))
            print('P@5: ', nn(5))
            print('P@10: ', nn(10))'''
     
    def calculate_precision(self, predictions_nn, actual):
        pos = 0
        total = len(actual)
        print("actual:", actual[0])
        print("pred:", predictions_nn[0][0])
        for (i, y) in enumerate(actual):
            if y in predictions_nn[i]:
                pos+=1
        print(str(pos) + "out of " + str(total))
        

In [ ]:
#k - number of negative
#gamma - margin diff from correct 
#pt-es